In [1]:
using LinearAlgebra 
using UnicodePlots 
using StaticArrays 
using SparseArrays
#Variables globales N,M,D
global N=3;
global M=3;
global D=Int((factorial(M+N-1))/(factorial(M-1)*factorial(N)))
#funcion primos 
#function p(i) 
#     return 100i + 3 
#end 
p(i)=100i+3
#Generamos la base en orden lexicografico para N=M=3
v = []; 
i=1;
for n1 in 0:N  
    for n2 in 0:(N-n1)  
        for n3 in 0:(N-n1-n2)  
            if n1+n2+n3 != N
                continue 
            end
            pushfirst!(v, [n1,n2,n3]) 
        end
    end
end   

#hacer un vector estatico 
#v = reverse(v)  
##v = SVector{D,Vector}(v)
# reverse!(v)  

In [2]:
function tag(b,M)
    tag=0;
    for i in 1:M
        tag += (√p(i))*b[i] 
    end
    return tag 
end

tag (generic function with 1 method)

In [3]:
function creation_ann(k,j,v)     
    if(v[j] >= 1) 
        b = copy(v)
        value = sqrt(((b[k])+1)*b[j])
        b[j] -= 1; 
        b[k] += 1;    
        return [value , b]   
    else 
        return 0; 
    end 
end   

creation_ann (generic function with 1 method)

In [8]:
#Valores de la diagonal de los numeros de ocupacion
function valdiags(k, v, M)  
    if(v[k] >= 1)  
        return  (v[k]*(v[k]-1))/2  
    else 
        return 0;
    end  
end       

valdiags (generic function with 1 method)

In [5]:
#Array de tags 
T = []
for i in 1:length(v) 
     push!(T,  tag( v[i] , M )) 
end

In [6]:
#Ordenamos y hacemos un diccionario Tsorted => indices
#sort!(T) 
#Tsorted = sort(T) 
inds = sortperm(T);
Tsorted = T[inds];
DictOfInds = Dict(zip(Tsorted, inds)) 
#Dict(inds .=> Tsorted)

Dict{Any, Int64} with 10 entries:
  30.4467 => 1
  34.5456 => 2
  42.7434 => 7
  44.9627 => 6
  41.8036 => 5
  49.0616 => 9
  38.6445 => 4
  37.7047 => 3
  45.9025 => 8
  52.2207 => 10

In [21]:
#for t in 1:10
#    J=10
#    U=t*J*0.1 
#obtener H_int 
    diagscomp = [];
    for j in 1:D 
        sumsdeocup = []; 
        for k in 1:M
        push!(sumsdeocup, valdiags(k, v[j], M))
        end  
    push!(diagscomp, sum(sumsdeocup))    
    end   
    #Mostramos la parte  
    println(diagscomp)
    Hint = spzeros(D,D)
    Hint[diagind(Hint)] = diagscomp; Hint 
    println(Hint)
    #Hint=diagm(diagscomp) 
    spy(Hint)
#end  

Any[3.0, 1.0, 1.0, 1.0, 0.0, 1.0, 3.0, 1.0, 1.0, 3.0]

 3.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   3.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   3.0


      ┌─────┐    
    1 │⠑⠤⠀⠀⠀│ > 0
   10 │⠀⠀⠈⠒⢄│ < 0
      └─────┘    
      ⠀1⠀⠀10⠀    
      9 nonzeros    

In [16]:
#obtener H_kin  
J = 5
Hkin = spzeros(D,D)
#println(DictOfInds[tag(creation_ann( 2, 1 , v[1])[2],M)])
#println(Dict1["b"])
#println(Dict1["c"]) 
#     J=10
#     #obtener H_kin
for k in 1:D
    for j in 1:M
     end 
end 
#             if v[k][j]==0
#                 continue 
#             end
#             if j <= (M-2)
#                 aa=creacion_aniquilacion(j+1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])
#             else
#                 aa=creacion_aniquilacion(0,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])   
#             end
#             if j>=1
#                 aa=creacion_aniquilacion(j-1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])
#             else
#                 aa=creacion_aniquilacion(M-1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])   
#             end
#         end 


2


In [9]:
#a = rand(2,2) 
#b = [1,2]   
#indices que pertenecen a la primera columna 
firstcolumn = []
for k in 1:M 
    for h in 1:M   
        if k == h 
            break 
        elseif creation_ann(k,h,v[1]) == 0 
            continue 
        else 
            push!(firstcolumn, DictOfInds[tag(creation_ann( k, h , v[1] )[2],M)] )   
        end
    end 
end 
println(firstcolumn) 
#DictOfInds[tag(creation_ann( 3, 1 , [3,0,0] )[2],M)] 
#creation_ann(3,1,v[1]) 
#creation_ann(1,3,v[1])  
#creation_ann(2,1,v[2]) 
#v

Any[2, 3]


In [16]:
J = 5
Hkin = spzeros(D,D)
firstcolumn = [] 
for j in 1:D
    for k in 1:M 
        for h in 1:M   
            if k == h 
                break 
            elseif creation_ann(k,h,v[j]) == 0 
                continue 
            else 
                 Hkin[DictOfInds[tag(creation_ann( k, h , v[j] )[2],M)], j] = -J*(creation_ann( k, h , v[j])[1])     
            end
        end 
    end  
end  
#Hkin = Symmetric(Hkin, :L)
println(Hkin)  
spy(Hkin)


   ⋅          ⋅          ⋅         ⋅         ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
 -8.66025     ⋅          ⋅         ⋅         ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
 -8.66025   -5.0         ⋅         ⋅         ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
   ⋅       -10.0         ⋅         ⋅         ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
   ⋅        -7.07107   -7.07107  -7.07107    ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
   ⋅          ⋅       -10.0        ⋅       -7.07107    ⋅         ⋅          ⋅     ⋅        ⋅ 
   ⋅          ⋅          ⋅       -8.66025    ⋅         ⋅         ⋅          ⋅     ⋅        ⋅ 
   ⋅          ⋅          ⋅       -5.0      -7.07107    ⋅       -8.66025     ⋅     ⋅        ⋅ 
   ⋅          ⋅          ⋅         ⋅       -7.07107  -5.0        ⋅       -10.0    ⋅        ⋅ 
   ⋅          ⋅          ⋅         ⋅         ⋅       -8.66025    ⋅          ⋅   -8.66025   ⋅ 


      ┌─────┐    
    1 │⢦⣀⠀⠀⠀│ > 0
   10 │⠀⠑⢧⠢⡀│ < 0
      └─────┘    
      ⠀1⠀⠀10⠀    
      18 nonzeros    

In [87]:
#creation_ann(2,1,v[1]) It works! 
v

10-element Vector{Any}:
 [3, 0, 0]
 [2, 1, 0]
 [2, 0, 1]
 [1, 2, 0]
 [1, 1, 1]
 [1, 0, 2]
 [0, 3, 0]
 [0, 2, 1]
 [0, 1, 2]
 [0, 0, 3]

In [10]:
# arr = [0,0,3,0];  
# arr[1];
# v[2] 
# function sumyprod(x,y) 
#     suma = x+y 
#     prod = x*y 
#     return suma, prod 
# end 
# for i in 1:10 
#     for j in 1:3
#         println(i, j, sumyprod(i , j) )
#     end 
# end

In [11]:
#creation_ann(1,2,v[2])
#tag(creation_ann(1,2,v[10]), M)  
#valdiags(10, v[10]) 
#v[2]

In [13]:
# for j in 1:10
#     cont = 0;
#     for i in 1:10  
#         cont+= i   
#         if(cont == 55)
#             println(cont, " ", j) 
#         end
#     end 
# end  
#valdiags(1, v[5], M)


In [14]:
#obtener H_int 
#     for t in 1:10
#     J=10
#     U=t*J*0.1
#     for k in 1:D
#         for j in range(M)
#             valoresH.append((U/2)*v[k][j]*(v[k][j]-1))
#             fila.append(k)
#             columna.append(k) 
#         end 
#     end

#     #obtener H_kin
#     for k in 1:D
#         for j in 1:M
#             if v[k][j]==0
#                 continue 
#             end
#             if j<=M-2
#                 aa=creacion_aniquilacion(j+1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])
#             else
#                 aa=creacion_aniquilacion(0,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])   
#             end
#             if j>=1
#                 aa=creacion_aniquilacion(j-1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])
#             else
#                 aa=creacion_aniquilacion(M-1,j,v,k)
#                 fila.append(aa[0])
#                 columna.append(k)
#                 valoresH.append((-J)*aa[1])   
#             end
#         end 
#     end 
# end
    
# row = np.array(fila)
# col = np.array(columna)
# data = np.array(valoresH)
# sparseMatrix = csr_matrix((data, (row, col)),  shape = (D, D))
# spy(sparseMatrix)

In [15]:
#Ejemplo sparce
A = sparse([1, 1, 2, 3], [1, 3, 2, 3], [0, 1, 2, 0])

3×3 SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 0  ⋅  1
 ⋅  2  ⋅
 ⋅  ⋅  0

In [16]:
spy(A)

     ┌─────┐    
   1 │⠀⠀⠀⠠⠀│ > 0
   3 │⠀⠀⠈⠀⠀│ < 0
     └─────┘    
     ⠀1⠀⠀⠀3⠀    
     2 nonzeros    